#### Gerardo Granados Aldaz   A01568072
#### Sebastian Salazar Villanueva  A01568158
#### Abraham Alejandro Saenz Tirado      A00823861

## Objetivo del proyecto: 
El propósito de este proyecto es poder elaborar un analisis exploratorio de
los libros empleando diversos algoritmos que estudiamos durante el curso. Analisis estadísticos,
algoritmos no supervisados y elementos de visualización serán sus principales herramientas para
este analisis. Adicionalmente cualquier algoritmo o herramienta que no hayamos visto durante el
curso es válida, sin embargo, es importante que conozcan el funcionamiento y puedan explicar lo
que se está realizando con claridad.

## Resultados del proyecto:
Los elementos mínimos que su analisis debe cumplir son los siguientes:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Para poder analizar los datos preprocesamos los textos a un .csv con los campos "campo: valor" antes de *** START OF THE PROJECT GUTENBERG EBOOK <<TITLE>> **
import glob
import regex as re
import requests

def create_csv(csv_name: str):
    # Dataframe para los libros
    df = pd.DataFrame(columns = ["Filename", "ID", "Title", "Author", "Language", "Release Date", "Subjects"])

    libros = glob.glob('./Gutenberg/*.txt')
    n = len(libros)

    # Iteramos por cada .txt
    for i, filename in enumerate(libros):
        print(f"Progress: {i:03d}/{n}    {i/n*100:.2f}%", end="\r")
        row = [filename]
        features = {"Title": None, "Author": None, "Language": None, "Release Date": None}

        with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
            # Itera sobre cada linea de texto
            for line in f:
                # Itera sobre las features excepto "Release Date"
                for feat in set(features) - {"Release Date"}:

                    match = re.search(fr"(?<={feat}:\s).*(?=$)", line)
                    if match:
                        features[feat] = match.group(0)

                # Matchea Release Date
                match = re.search(r"(?<=Release Date:\s).*(?=\s\[.*\]$)", line)
                if match:
                    features["Release Date"] = match.group(0)

                # Si ya tenemos todos los features, dejar de iterar por lineas de texto
                if None not in features.values():
                    break
                
                
        # Conseguir generos
        id = re.search(r'((?<!-)((?<=(pg)*)|(?<=^)))\d*((?=.txt)|(?=-0.txt))' ,filename).group(0)
        
        row.append(id)
        
        row.extend(features.values())
        
        # Subjects
        # Hacemos un request a gutendex, una API para el Proyecto Gutenberg para sacar los subjects
        r = requests.get(f'https://gutendex.com/books/{id}')
        subjects = r.json().get('subjects')
        # print(subjects)
        
        row.append(subjects)
        
        if None not in row:
            df.loc[len(df)] = row

    df.set_index("Filename")
    df.to_csv(csv_name, index=False)
    return df

In [3]:
import os.path

fname = 'gutenberg.csv'

# importar de csv si ya existe
if os.path.isfile(fname):
    df = pd.read_csv(fname)
else:
    df = create_csv(fname)
    
df

,Filename,ID,Title,Author,Language,Release Date,Subjects
0,./Gutenberg\24747-0.txt,24747,Samlede værker,Jeppe Aakjær,Danish,"March 3, 2008",['Danish poetry']
1,./Gutenberg\27566-0.txt,27566,La Force,Paul Adam,French,"December 19, 2008","['Historical fiction', 'Napoleonic Wars, 1800-..."
2,./Gutenberg\33378-0.txt,33378,Au soleil de juillet (1829-1830),Paul Adam,French,"August 8, 2010","['France -- History -- July Revolution, 1830 -..."
3,./Gutenberg\36460-0.txt,36460,Cœur-de-panthère,Gustave Aimard,French,"June 18, 2011","['Adventure and adventurers -- Fiction', 'West..."
4,./Gutenberg\39331-0.txt,39331,Histoire des salons de Paris (Tome 1/6),"Laure Junot, duchesse d'Abrantès",French,"April 1, 2012","['Paris (France) -- Intellectual life', 'Paris..."
...,...,...,...,...,...,...,...
192,./Gutenberg\pg70639.txt,70639,"The Life and Adventures of Alexander Selkirk, ...",John Howell,English,"April 24, 2023","['Defoe, Daniel, 1661?-1731. Robinson Crusoe',..."
193,./Gutenberg\pg70640.txt,70640,Third Base Thatcher,Everett (Deacon) Scott,English,"April 24, 2023","['Baseball -- Juvenile fiction', 'Baseball sto..."
194,./Gutenberg\pg70642.txt,70642,"Historical Vignettes, 1st Series",Bernard Capes,English,"April 25, 2023","['Historical fiction', 'Short stories, English']"
195,./Gutenberg\pg70643.txt,70643,Coca and Cocaine,William Martindale,English,"April 26, 2023","['Coca', 'Cocaine']"


In [4]:
# Separamos libros por idioma
languages = df['Language'].unique()

# Diccionario con los dataframes por idioma
df_langs = {}
for lang in languages:
    new_df = df[df['Language'] == lang]
    df_langs[lang] = new_df
    print(lang, len(new_df))

Danish 3
French 57
Spanish 86
English 47
English and Spanish 2
Iloko, Spanish 2


In [5]:
# Combinamos los generos a una sola lista y ordenamos por tamaño

subjects = sorted(list(set([a for b in df.Subjects.tolist() for a in b])), key=len)

subjects, len(subjects)

(['x',
  '8',
  'e',
  's',
  'i',
  '[',
  'F',
  'd',
  'V',
  'U',
  'é',
  'Y',
  '9',
  'b',
  '6',
  'w',
  'N',
  'm',
  'n',
  ',',
  'c',
  'f',
  'z',
  'R',
  'J',
  'B',
  'L',
  'Q',
  'E',
  'h',
  'í',
  '0',
  't',
  '5',
  'W',
  'k',
  'T',
  'p',
  'G',
  '"',
  '.',
  'S',
  'á',
  ':',
  ']',
  'v',
  'X',
  'l',
  'P',
  '2',
  'A',
  'a',
  'H',
  '3',
  'C',
  'y',
  'q',
  'u',
  'D',
  '?',
  '4',
  '1',
  "'",
  '-',
  '(',
  'M',
  'o',
  'g',
  'I',
  ')',
  '7',
  'O',
  'ó',
  'r',
  ' ',
  'K'],
 76)

In [6]:
# Estan sucios los datos con generos demasiado largos y especificos, estos generos tienen '--' vamos a quitar esos generos

subjects = [sub for sub in subjects if '--' not in sub and '(' not in sub]
subjects

['x',
 '8',
 'e',
 's',
 'i',
 '[',
 'F',
 'd',
 'V',
 'U',
 'é',
 'Y',
 '9',
 'b',
 '6',
 'w',
 'N',
 'm',
 'n',
 ',',
 'c',
 'f',
 'z',
 'R',
 'J',
 'B',
 'L',
 'Q',
 'E',
 'h',
 'í',
 '0',
 't',
 '5',
 'W',
 'k',
 'T',
 'p',
 'G',
 '"',
 '.',
 'S',
 'á',
 ':',
 ']',
 'v',
 'X',
 'l',
 'P',
 '2',
 'A',
 'a',
 'H',
 '3',
 'C',
 'y',
 'q',
 'u',
 'D',
 '?',
 '4',
 '1',
 "'",
 '-',
 'M',
 'o',
 'g',
 'I',
 ')',
 '7',
 'O',
 'ó',
 'r',
 ' ',
 'K']

In [7]:
# Siguen sucios con comas, ahora dejamos solo la primera parte de los generos con coma y quitamos duplicados
subjects = list(set([sub.split(',')[0] for sub in subjects]))
subjects

['',
 'x',
 '8',
 'e',
 's',
 'i',
 '[',
 'F',
 'V',
 'U',
 'é',
 'Y',
 '9',
 'b',
 'w',
 '6',
 'N',
 'm',
 'n',
 'z',
 'c',
 'f',
 'B',
 'R',
 'J',
 'Q',
 'L',
 ' ',
 'E',
 'h',
 'í',
 '0',
 't',
 '5',
 'W',
 'k',
 'T',
 'p',
 'G',
 '"',
 '.',
 'S',
 'á',
 ':',
 ']',
 'v',
 'X',
 'l',
 'P',
 '2',
 'A',
 'a',
 'H',
 '3',
 'C',
 'y',
 'q',
 'u',
 'D',
 '?',
 '4',
 '1',
 "'",
 '-',
 'M',
 '7',
 'o',
 'g',
 'I',
 ')',
 'O',
 'ó',
 'r',
 'd',
 'K']

In [8]:
# Ya limpios podemos decidir los generos manualmente
genres = ['Fiction',
 'Fairy tales',
 'Political fiction',
 'Detective and mystery stories',
 'Mythology',
 'Sainetes',
 'Ballads',
 'Fantasy fiction',
 'Classical literature',
 'Christmas stories',
 'Speeches',
 'Short stories',
 'Cooking',
 'Baseball stories',
 'Picaresque literature',
 'Farces',
 'Historical fiction',
 'Domestic fiction',
 'Horror tales',
 'War stories',
 'Comedies',
 'Christian fiction',
 'Science fiction',
 'Poetry',
 'Spanish drama',
 'Love stories',
 'Humorous stories',
 'Adventure stories',
 'Psychological fiction']

## 1. Contestar los siguientes cuestionamientos
### a. ¿Cuántos y cuales son los idiomas que identificaron en los libros?
5 Idiomas: Danés, Francés, Español, Inglés e Iloco. Unos estan en Ingles y Español y otros en Iloco y Español

### b. ¿Cuántos libros tenemos por cada idioma?
Danés: 3<br>
Francés: 57<br>
Español: 86<br>
Inglés: 47<br>
Ingles y Español: 2<br>
Iloco y Español: 2<br>

### c. ¿Cuántos géneros tenemos? y ¿Qué proceso utilizaron para identificar el (los) géneros del libro?
Conseguimos los generos con la ayuda de la API gutendex.com. Hicimos un request para sacar los subjects de cada libro<br>
Originalmente eran 304 temas diferentes, pero despues de limpiar, quedamos con 29 géneros que decidimos basados en los temas existentes

### ¿Encontraron algún otro patrón o elemento que consideren importante mencionar en su analisis?
Los temas eran demasiado especificos, incluian autores y años. Estos venían separados por un doble guión y tenían paréntesis. Otros incluían el idioma en el tema y genero, esos los eliminamos como duplicados porque tenemos el dato en otras partes del dataset.

2. Elemento de visualización<br>
    a. ¿Cuáles son las palabras con mayor frecuencia en cada idioma?<br>
    b. ¿Es posible agrupar los libros mediante cluster de tal forma que se ilustren los
    grupos por idioma?<br>
        i. De ser así genera una gráfica que ilustre esto<br>
        ii. Caso contario explica el motivo<br>
    c. Genera una grafica que ilustre la identificación de géneros en los libros